## 시계열을 위한 딥러닝

### 다양한 종류의 시계열 작업

#### 시계열(timeseries) 데이터
- 일정한 간격으로 측정하여 얻은 모든 데이터
- 예. 주식의 일별 가격, 도시의 시간별 전력 소모량, 상점의 주간별 판매량 등
- 자연 현상 : 지진 활동, 강에 있는 물고기 개체 수의 변화, 한 지역의 날씨
- 인간의 활동 패턴 : 웹 사이트 방문자, 국가의 GDP, 신용 카드 거래
- 주기성, 시간에 따른 트렌드, 규칙적인 형태와 급격한 증가 등

#### 시계열 관련 작업

- 예측(forecasting) : 현시점의 시계열 데이터 다음에 일어날 것을 예측
    - 몇 시간 후 전력 소모량 예측 → 수요 예상
    - 몇 달 후 수익 예측 → 예산 계획
    - 며칠 후 날씨 예측 → 일정 계획
- 분류 : 하나 이상의 범주형 레이블을 시계열에 부여
    - 웹 사이트의 방문자 활동에 대한 시계열 → 이 방문자가 봇(bot)인지 사람인지 분류
- 이벤트 감지 : 연속된 데이터 스트림에서 예상되는 특정 이벤트 발생을 식별
    - 핫워드(hotword) 감지 : 모델이 오디오 스트림을 모니터링하다가 ‘오케이 구글’ 또는 ‘안녕 알레사’와 같은 시작 단어를 감지
- 이상치 탐지(anomaly detection): 연속된 데이터 스트림에서 발생하는 비정상적인 현상을 감지
    - 회사 네트워크에 비정상적인 활동이 있다면 외부 공격일 수 있음
    - 제조 라인에 비정상적인 값이 측정되었다면 사람이 가서 확인해 보아야 함
    - 비지도 학습(unsupervised learning)으로 수행
        - 어떤 종류의 이상치를 찾는지 모르는 경우가 많아 구체적인 이상치 샘플로 훈련할 수 없기 때문임

### 기온 예측 문제

#### 예제 데이터
- 독일 예나(Jena)시에 이는 막스 플랑크 샐물지구화학연구소의 기상관측소에서 수집한 기상 데이터
- 14개 요인(온도, 기압, 습도, 풍향 등)에 대하여 10분 마다 기록된 2009~2016년 사이의 데이터
- 데이터 다운로드 및 압축풀기
- https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip

In [ ]:
!curl -O https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip

#### 날씨 데이터셋 로딩

#### 피처데이터와 타깃데이터 

- features : 'Date Time' 컬럼은 제외하고, 나머지 컬럼들의 데이터(온도 포함)
- targets : 'T (degC)' 컬럼(온도)

#### 전체 기온 시각화

### 데이터 준비

- 데이터는 3D
- 입력 형태 : (samples, timesteps, features)
    - samples: 전체 시계열 샘플의 수 (ex. 1000개 샘플)
    - timesteps: 한 샘플에 포함된 시간의 길이 (ex. 과거 10일간의 데이터)
    - features: 각 시간단계에서의 특성 수 (ex. 온도, 습도 등 2개 변수)

**데이터 정규화**

**keras.utils.timeseries_dataset_from_array()**

- 시계열 데이터를 모델 학습용 시퀀스 데이터로 자동 변환해주는 함수
- 주로 LSTM, GRU, RNN 모델에 입력할 데이터를 쉽게 준비해줌
```python
tf.keras.utils.timeseries_dataset_from_array(
    data,
    targets,
    sequence_length,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=128,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None
)

#### 훈련, 검증, 테스트 데이터셋 만들기

**각 분할에 사용할 샘플 수 계산하기**

**훈련 데이터셋의 배치 크기 확인하기**

**데이터소스 지정**

### 상식 수준의 기준점

**상식적인 기준 모델의 MAE 계산하기**

### 기본적인 머신 러닝 모델 시도해 보기

**밀집 연결 모델 훈련하고 평가하기**

### 1D 합성곱 모델 시도해 보기

### 첫 번째 순환 신경망

**간단한 LSTM 기반 모델**

**넘파이로 구현한 간단한 RNN**

In [19]:
import numpy as np
timesteps = 100
input_features = 32
output_features = 64
inputs = np.random.random((timesteps, input_features))
state_t = np.zeros((output_features,))
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))
successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t = output_t
final_output_sequence = np.stack(successive_outputs, axis=0)

### 케라스의 순환 층

**어떤 길이의 시퀀스도 처리할 수 있는 RNN 층**

In [20]:
num_features = 14
inputs = keras.Input(shape=(None, num_features))
outputs = layers.SimpleRNN(16)(inputs)

**마지막 출력 스텝만 반환하는 RNN 층**

In [21]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=False)(inputs)
print(outputs.shape)

(None, 16)


**전체 출력 시퀀스를 반환하는 RNN 층**

In [22]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs)
print(outputs.shape)

(None, 120, 16)


**스태킹(stacking) RNN 층**

In [23]:
inputs = keras.Input(shape=(steps, num_features))
x = layers.SimpleRNN(16, return_sequences=True)(inputs)
x = layers.SimpleRNN(16, return_sequences=True)(x)
outputs = layers.SimpleRNN(16)(x)

### 과대적합을 감소하기 위해 순환 드롭아웃 사용하기

**드롭아웃 규제를 적용한 LSTM 모델 훈련하고 평가하기**

### 스태킹(다층) 순환 층

**드롭아웃 규제와 스태킹을 적용한 GRU 모델을 훈련하고 평가하기**

---